<a href="https://www.kaggle.com/code/mahendrasinghrajpoot/building-a-recommendation-system?scriptVersionId=96953961" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from pandas import Series,DataFrame
from sklearn.linear_model import LogisticRegression
from sklearn.decomposition import TruncatedSVD
from sklearn.neighbors import NearestNeighbors
from sklearn.metrics import classification_report

In [2]:
#loading the file into the environment
results = pd.read_csv("../input/restaurant-data-with-consumer-ratings/rating_final.csv")
cuisine = pd.read_csv("../input/restaurant-data-with-consumer-ratings/chefmozcuisine.csv")

In [3]:
results.head()

,userID,placeID,rating,food_rating,service_rating
0,U1077,135085,2,2,2
1,U1077,135038,2,2,1
2,U1077,132825,2,2,2
3,U1077,135060,1,2,2
4,U1068,135104,1,1,2


In [4]:
cuisine.head()

,placeID,Rcuisine
0,135110,Spanish
1,135109,Italian
2,135107,Latin_American
3,135106,Mexican
4,135105,Fast_Food


In [5]:
rating_count = pd.DataFrame(results.groupby('placeID')['rating'].count())

In [6]:
rating_count.sort_values('rating',ascending=False).head()

,rating
placeID,
135085,36
132825,32
135032,28
135052,25
132834,25


In [7]:
#so we have extracted this information from the above table and built the dataframe to merge with other column
most_rated_places=pd.DataFrame([135085,132825,135032,135052,132834],index=np.arange(5),columns=['placeID'])

In [8]:
summary = pd.merge(most_rated_places,cuisine,on='placeID')
summary

,placeID,Rcuisine
0,135085,Fast_Food
1,132825,Mexican
2,135032,Cafeteria
3,135032,Contemporary
4,135052,Bar
5,135052,Bar_Pub_Brewery
6,132834,Mexican


In [9]:
cuisine['Rcuisine'].describe()
#we can see the mexican cuisine is pretty famous

count         916
unique         59
top       Mexican
freq          239
Name: Rcuisine, dtype: object

In [10]:
#Recommendation system based on correlation ,we will be using the pearson;s correlation to recommend an item that is most similar to the item a user has already chosen
#we will be taking the help of the geodata, to find the popularity based on the demographic 
geodata = pd.read_csv("../input/restaurant-data-with-consumer-ratings/geoplaces2.csv")
geodata.head()

,placeID,latitude,longitude,the_geom_meter,name,address,city,state,country,fax,...,alcohol,smoking_area,dress_code,accessibility,price,url,Rambience,franchise,area,other_services
0,134999,18.915421,-99.184871,0101000020957F000088568DE356715AC138C0A525FC46...,Kiku Cuernavaca,Revolucion,Cuernavaca,Morelos,Mexico,?,...,No_Alcohol_Served,none,informal,no_accessibility,medium,kikucuernavaca.com.mx,familiar,f,closed,none
1,132825,22.147392,-100.983092,0101000020957F00001AD016568C4858C1243261274BA5...,puesto de tacos,esquina santos degollado y leon guzman,s.l.p.,s.l.p.,mexico,?,...,No_Alcohol_Served,none,informal,completely,low,?,familiar,f,open,none
2,135106,22.149709,-100.976093,0101000020957F0000649D6F21634858C119AE9BF528A3...,El Rinc�n de San Francisco,Universidad 169,San Luis Potosi,San Luis Potosi,Mexico,?,...,Wine-Beer,only at bar,informal,partially,medium,?,familiar,f,open,none
3,132667,23.752697,-99.163359,0101000020957F00005D67BCDDED8157C1222A2DC8D84D...,little pizza Emilio Portes Gil,calle emilio portes gil,victoria,tamaulipas,?,?,...,No_Alcohol_Served,none,informal,completely,low,?,familiar,t,closed,none
4,132613,23.752903,-99.165076,0101000020957F00008EBA2D06DC8157C194E03B7B504E...,carnitas_mata,lic. Emilio portes gil,victoria,Tamaulipas,Mexico,?,...,No_Alcohol_Served,permitted,informal,completely,medium,?,familiar,t,closed,none


In [11]:
geodata.shape

(130, 21)

In [12]:
#selecting only two column from the dataset we have 
places= geodata[['placeID','name']]
places.head()

,placeID,name
0,134999,Kiku Cuernavaca
1,132825,puesto de tacos
2,135106,El Rinc�n de San Francisco
3,132667,little pizza Emilio Portes Gil
4,132613,carnitas_mata


In [13]:
cuisine.head()

,placeID,Rcuisine
0,135110,Spanish
1,135109,Italian
2,135107,Latin_American
3,135106,Mexican
4,135105,Fast_Food


In [14]:
rating_mean = pd.DataFrame(results.groupby('placeID')['rating'].mean())
rating_mean.head()

,rating
placeID,
132560,0.50
132561,0.75
132564,1.25
132572,1.00
132583,1.00


In [15]:
rating_mean['rating_count']=pd.DataFrame(rating_count)
rating_mean.head()

,rating,rating_count
placeID,,
132560,0.50,4
132561,0.75,4
132564,1.25,4
132572,1.00,15
132583,1.00,4


In [16]:
#finding the statistical descriptions
rating_mean.describe()

,rating,rating_count
count,130.000000,130.000000
mean,1.179622,8.930769
std,0.349354,6.124279
min,0.250000,3.000000
25%,1.000000,5.000000
50%,1.181818,7.000000
75%,1.400000,11.000000
max,2.000000,36.000000


In [17]:
#sorting the place in the correct order
rating_mean.sort_values('rating_count',ascending=False).head()

,rating,rating_count
placeID,,
135085,1.333333,36
132825,1.281250,32
135032,1.178571,28
135052,1.280000,25
132834,1.000000,25


In [18]:
places[places['placeID'] == 135085]

,placeID,name
121,135085,Tortas Locas Hipocampo


In [19]:
cuisine[cuisine['placeID']==135085]

,placeID,Rcuisine
44,135085,Fast_Food


In [20]:
places_crosstab = pd.pivot_table(data=results,values='rating',index='userID',columns='placeID')
places_crosstab.head()

placeID,132560,132561,132564,132572,132583,132584,132594,132608,132609,132613,...,135080,135081,135082,135085,135086,135088,135104,135106,135108,135109
userID,,,,,,,,,,,,,,,,,,,,,
U1001,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN
U1002,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,1.0,NaN,NaN,NaN,1.0,NaN,NaN
U1003,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
U1004,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,NaN,NaN
U1005,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [21]:
tortas_ratings = places_crosstab[135085]
tortas_ratings[tortas_ratings>=0]

userID
U1001    0.0
U1002    1.0
U1007    1.0
U1013    1.0
U1016    2.0
U1027    1.0
U1029    1.0
U1032    1.0
U1033    2.0
U1036    2.0
U1045    2.0
U1046    1.0
U1049    0.0
U1056    2.0
U1059    2.0
U1062    0.0
U1077    2.0
U1081    1.0
U1084    2.0
U1086    2.0
U1089    1.0
U1090    2.0
U1092    0.0
U1098    1.0
U1104    2.0
U1106    2.0
U1108    1.0
U1109    2.0
U1113    1.0
U1116    2.0
U1120    0.0
U1122    2.0
U1132    2.0
U1134    2.0
U1135    0.0
U1137    2.0
Name: 135085, dtype: float64

# Evaluating Similarity Based on Correlation

In [22]:
similar_to_tortas = places_crosstab.corrwith(tortas_ratings)

/opt/conda/lib/python3.7/site-packages/numpy/lib/function_base.py:2683: RuntimeWarning: Degrees of freedom <= 0 for slice
  c = cov(x, y, rowvar, dtype=dtype)
/opt/conda/lib/python3.7/site-packages/numpy/lib/function_base.py:2542: RuntimeWarning: divide by zero encountered in true_divide
  c *= np.true_divide(1, fact)


In [23]:
corr_tortas = pd.DataFrame(similar_to_tortas,columns=['PearsonR'])
corr_tortas.dropna(inplace=True)
corr_tortas.head()

,PearsonR
placeID,
132572,-0.428571
132723,0.301511
132754,0.930261
132825,0.700745
132834,0.814823


In [24]:
tortas_corr_summary = corr_tortas.join(rating_mean['rating_count'])
tortas_corr_summary[tortas_corr_summary['rating_count']>=10].sort_values('PearsonR',ascending=False).head(10)

,PearsonR,rating_count
placeID,,
135076,1.000000,13
135085,1.000000,36
135066,1.000000,12
132754,0.930261,13
135045,0.912871,13
135062,0.898933,21
135028,0.892218,15
135042,0.881409,20
135046,0.867722,11


In [25]:
places_corr_tortas = pd.DataFrame([135085,132745,135062,135045,135028,135042,135046],index=np.arange(7),columns=['placeID'])
summary = pd.merge(places_corr_tortas,cuisine,on='placeID')
summary

,placeID,Rcuisine
0,135085,Fast_Food
1,132745,American
2,135028,Mexican
3,135042,Chinese
4,135046,Fast_Food


In [26]:
places[places['placeID']==135046]

,placeID,name
42,135046,Restaurante El Reyecito


In [27]:
cuisine['Rcuisine'].describe()

count         916
unique         59
top       Mexican
freq          239
Name: Rcuisine, dtype: object

# Classification Based Collaborative Filtering

In [28]:
bank_full = pd.read_csv("../input/banking-dataset/bank_full_w_dummy_vars.csv")
bank_full.head()

,age,job,marital,education,default,balance,housing,loan,contact,day,...,job_unknown,job_retired,job_services,job_self_employed,job_unemployed,job_maid,job_student,married,single,divorced
0,58,management,married,tertiary,no,2143,yes,no,unknown,5,...,0,0,0,0,0,0,0,1,0,0
1,44,technician,single,secondary,no,29,yes,no,unknown,5,...,0,0,0,0,0,0,0,0,1,1
2,33,entrepreneur,married,secondary,no,2,yes,yes,unknown,5,...,0,0,0,0,0,0,0,1,0,0
3,47,blue-collar,married,unknown,no,1506,yes,no,unknown,5,...,0,0,0,0,0,0,0,1,0,0
4,33,unknown,single,unknown,no,1,no,no,unknown,5,...,1,0,0,0,0,0,0,0,1,1


In [29]:
bank_full.shape

(45211, 37)

In [30]:
bank_full.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45211 entries, 0 to 45210
Data columns (total 37 columns):
 #   Column                        Non-Null Count  Dtype 
---  ------                        --------------  ----- 
 0   age                           45211 non-null  int64 
 1   job                           45211 non-null  object
 2   marital                       45211 non-null  object
 3   education                     45211 non-null  object
 4   default                       45211 non-null  object
 5   balance                       45211 non-null  int64 
 6   housing                       45211 non-null  object
 7   loan                          45211 non-null  object
 8   contact                       45211 non-null  object
 9   day                           45211 non-null  int64 
 10  month                         45211 non-null  object
 11  duration                      45211 non-null  int64 
 12  campaign                      45211 non-null  int64 
 13  pdays           

In [31]:
bank_full.isna().sum()

age                             0
job                             0
marital                         0
education                       0
default                         0
balance                         0
housing                         0
loan                            0
contact                         0
day                             0
month                           0
duration                        0
campaign                        0
pdays                           0
previous                        0
poutcome                        0
y                               0
y_binary                        0
housing_loan                    0
credit_in_default               0
personal_loans                  0
prev_failed_to_subscribe        0
prev_subscribed                 0
job_management                  0
job_tech                        0
job_entrepreneur                0
job_bluecollar                  0
job_unknown                     0
job_retired                     0
job_services  

In [32]:
#we are going to select the some of the columns and then we will going to feed it into the model that we are going to create 
X = bank_full.iloc[:,[18,19,20,21,22,23,24,25,26,27,28,29,31,32,33,34,35,36]].values

In [33]:
y = bank_full.iloc[:,17].values

In [34]:
logreg = LogisticRegression()
logreg.fit(X,y)

LogisticRegression()

In [35]:
new_user = [0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1]
#new_user = new_user.reshape(-1,1)
#need to reshape the array before feeding it to the model,below code is the best way to do it.
X2D = np.atleast_2d(new_user)

In [36]:
#now we will going to predict it based on the data that we have provided to the model
y_pred = logreg.predict(X2D)
y_pred
#it means that according to our model, he will accept the offer

array([1])

# Model - Based Collaborative Filtering

In [37]:
#we will be going to se the utility matrix and the singular value decomposition 
#a SVD is the linear algebra method that can decompose a utility matrix into three compressed matrix
#it is used in the model based recommendation system because we use this compressed matrix to make recommendation
#without having to refer back to the complete dataset

#in this we will also going to discuss the latent variables,which are present within and affects the behavior of the dataset
#the formula for it is A = u * S * v
#where u is left orthogonal, S is the diagonal matrix, and v is the right orthogonal


In [38]:
#importing the dataset into the environment
columns = ['user_id','Item_id','rating','timestamp']
frame = pd.read_csv("../input/movie-dataset-1/ml-100k/u.data",sep='\t',names=columns)


In [39]:
frame.head(5)

,user_id,Item_id,rating,timestamp
0,196,242,3,881250949
1,186,302,3,891717742
2,22,377,1,878887116
3,244,51,2,880606923
4,166,346,1,886397596


In [40]:
frame.shape

(100000, 4)

In [41]:
columns = ['Item_id','movie_title','release_date','video_release_date','IMDB_url',
          'unknown' , 'Action' , 'Adventure' , 'Animation',
          'Childrens' , 'Comedy' , 'Crime' , 'Documentary' , 'Drama','Fantasy',
              'Film-Noir' , 'Horror' , 'Musical' , 'Mystery' , 'Romance' , 'Sci-Fi' ,
              'Thriller' , 'War' , 'Western' ]
movies = pd.read_csv("../input/movie-dataset-1/ml-100k/u.item",sep='|',names=columns,encoding = 'latin-1')

In [42]:
movies.head()

,Item_id,movie_title,release_date,video_release_date,IMDB_url,unknown,Action,Adventure,Animation,Childrens,...,Fantasy,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
0,1,Toy Story (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Toy%20Story%2...,0,0,0,1,1,...,0,0,0,0,0,0,0,0,0,0
1,2,GoldenEye (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?GoldenEye%20(...,0,1,1,0,0,...,0,0,0,0,0,0,0,1,0,0
2,3,Four Rooms (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Four%20Rooms%...,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
3,4,Get Shorty (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Get%20Shorty%...,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,5,Copycat (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Copycat%20(1995),0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0


In [43]:
movies.shape

(1682, 24)

In [44]:
movies.isna().sum()

Item_id                  0
movie_title              0
release_date             1
video_release_date    1682
IMDB_url                 3
unknown                  0
Action                   0
Adventure                0
Animation                0
Childrens                0
Comedy                   0
Crime                    0
Documentary              0
Drama                    0
Fantasy                  0
Film-Noir                0
Horror                   0
Musical                  0
Mystery                  0
Romance                  0
Sci-Fi                   0
Thriller                 0
War                      0
Western                  0
dtype: int64

In [45]:
combined_movies_data = pd.merge(frame,movies,on='Item_id')
combined_movies_data.head()

,user_id,Item_id,rating,timestamp,movie_title,release_date,video_release_date,IMDB_url,unknown,Action,...,Fantasy,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
0,196,242,3,881250949,Kolya (1996),24-Jan-1997,NaN,http://us.imdb.com/M/title-exact?Kolya%20(1996),0,0,...,0,0,0,0,0,0,0,0,0,0
1,63,242,3,875747190,Kolya (1996),24-Jan-1997,NaN,http://us.imdb.com/M/title-exact?Kolya%20(1996),0,0,...,0,0,0,0,0,0,0,0,0,0
2,226,242,5,883888671,Kolya (1996),24-Jan-1997,NaN,http://us.imdb.com/M/title-exact?Kolya%20(1996),0,0,...,0,0,0,0,0,0,0,0,0,0
3,154,242,3,879138235,Kolya (1996),24-Jan-1997,NaN,http://us.imdb.com/M/title-exact?Kolya%20(1996),0,0,...,0,0,0,0,0,0,0,0,0,0
4,306,242,5,876503793,Kolya (1996),24-Jan-1997,NaN,http://us.imdb.com/M/title-exact?Kolya%20(1996),0,0,...,0,0,0,0,0,0,0,0,0,0


In [46]:
#now finding out the most amount of data set that is rated by the movie goers and try to link them together
combined_movies_data.groupby('Item_id')['rating'].count().sort_values(ascending=False).head()

Item_id
50     583
258    509
100    508
181    507
294    485
Name: rating, dtype: int64

In [47]:
#lets check out the most reviewed movie in the datset and find out its info regarding itF
filter_1 = combined_movies_data['Item_id']==50
combined_movies_data[filter_1]['movie_title'].unique()
#as we can see that the most reviewed movie on the imbd is the stars wars

array(['Star Wars (1977)'], dtype=object)

# Building a Utility Matrix

In [48]:
#so here we are going to create the utility matrix and turn it into the transpose to find the SVD
#this is the first step in creating the svd of our dataset
rating_crosstab = combined_movies_data.pivot_table(values='rating',index='user_id',columns='movie_title',fill_value=0)
rating_crosstab.head()

movie_title,'Til There Was You (1997),1-900 (1994),101 Dalmatians (1996),12 Angry Men (1957),187 (1997),2 Days in the Valley (1996),"20,000 Leagues Under the Sea (1954)",2001: A Space Odyssey (1968),3 Ninjas: High Noon At Mega Mountain (1998),"39 Steps, The (1935)",...,Yankee Zulu (1994),Year of the Horse (1997),You So Crazy (1994),Young Frankenstein (1974),Young Guns (1988),Young Guns II (1990),"Young Poisoner's Handbook, The (1995)",Zeus and Roxanne (1997),unknown,Á köldum klaka (Cold Fever) (1994)
user_id,,,,,,,,,,,,,,,,,,,,,
1,0,0,2,5,0,0,3,4,0,0,...,0,0,0,5,3,0,0,0,4,0
2,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,2,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,0,0,2,0,0,0,0,4,0,0,...,0,0,0,4,0,0,0,0,4,0


> Transposing the Matrix

In [49]:
rating_crosstab.shape

(943, 1664)

In [50]:
x = rating_crosstab.values.T #transposing the matrix then we will be doing the SVD
x.shape    

(1664, 943)

> Decomposing the Matrix

In [51]:
SVD = TruncatedSVD(n_components=12,random_state=17)
#this fits the svd model and fits the dimensionality reduction in the model
resultant_matrix = SVD.fit_transform(x)
#finding the shape of the matrix
resultant_matrix.shape
#so we have reduced the 943 dimensional into the 12 variables

(1664, 12)

> Generating the correlation matrix

In [52]:
#let create the correlation matrix
corr_mat = np.corrcoef(resultant_matrix)
corr_mat.shape

(1664, 1664)

> Isolating the stars wars from this correlation matrix

In [53]:
movies_names = rating_crosstab.columns
movies_list = list(movies_names)

star_wars = movies_list.index("Star Wars (1977)")
print(star_wars)

1398


In [54]:
corr_star_wars = corr_mat[star_wars]
corr_star_wars.shape

(1664,)

> Recommending a highly Correlated Movie

In [55]:
list(movies_names[(corr_star_wars < 1.0) & (corr_star_wars > 0.9)])
#this movies has a very high correlation with the matrix

['Die Hard (1988)',
 'Empire Strikes Back, The (1980)',
 'Fugitive, The (1993)',
 'Raiders of the Lost Ark (1981)',
 'Return of the Jedi (1983)',
 'Terminator 2: Judgment Day (1991)',
 'Terminator, The (1984)',
 'Toy Story (1995)']

In [56]:
list(movies_names[(corr_star_wars < 1.0) & (corr_star_wars > 0.95)])
#this shows movie that are very closely related to the stars wars

['Return of the Jedi (1983)']

# Content-Based Recommender System

In [57]:
cars = pd.read_csv('../input/mtcars/mtcars.csv')
cars.head()

,Unnamed: 0,mpg,cyl,disp,hp,drat,wt,qsec,vs,am,gear,carb
0,Mazda RX4,21.0,6,160.0,110,3.90,2.620,16.46,0,1,4,4
1,Mazda RX4 Wag,21.0,6,160.0,110,3.90,2.875,17.02,0,1,4,4
2,Datsun 710,22.8,4,108.0,93,3.85,2.320,18.61,1,1,4,1
3,Hornet 4 Drive,21.4,6,258.0,110,3.08,3.215,19.44,1,0,3,1
4,Hornet Sportabout,18.7,8,360.0,175,3.15,3.440,17.02,0,0,3,2


In [58]:
cars.rename({'Unnamed: 0':'cars_name'}, axis=1)

,cars_name,mpg,cyl,disp,hp,drat,wt,qsec,vs,am,gear,carb
0,Mazda RX4,21.0,6,160.0,110,3.90,2.620,16.46,0,1,4,4
1,Mazda RX4 Wag,21.0,6,160.0,110,3.90,2.875,17.02,0,1,4,4
2,Datsun 710,22.8,4,108.0,93,3.85,2.320,18.61,1,1,4,1
3,Hornet 4 Drive,21.4,6,258.0,110,3.08,3.215,19.44,1,0,3,1
4,Hornet Sportabout,18.7,8,360.0,175,3.15,3.440,17.02,0,0,3,2
5,Valiant,18.1,6,225.0,105,2.76,3.460,20.22,1,0,3,1
6,Duster 360,14.3,8,360.0,245,3.21,3.570,15.84,0,0,3,4
7,Merc 240D,24.4,4,146.7,62,3.69,3.190,20.00,1,0,4,2
8,Merc 230,22.8,4,140.8,95,3.92,3.150,22.90,1,0,4,2
9,Merc 280,19.2,6,167.6,123,3.92,3.440,18.30,1,0,4,4


In [59]:
t = [15,300,160,3.2]

x = cars.iloc[:,[1,3,4,6]].values
x[0:5]

array([[ 21.   , 160.   , 110.   ,   2.62 ],
       [ 21.   , 160.   , 110.   ,   2.875],
       [ 22.8  , 108.   ,  93.   ,   2.32 ],
       [ 21.4  , 258.   , 110.   ,   3.215],
       [ 18.7  , 360.   , 175.   ,   3.44 ]])

In [60]:
nbrs = NearestNeighbors(n_neighbors=1).fit(x)

In [61]:
print(nbrs.kneighbors([t]))

(array([[10.77474942]]), array([[22]]))


# Evaluating the Recommendataion system

In [62]:
bank_full = pd.read_csv("../input/banking-dataset/bank_full_w_dummy_vars.csv")
bank_full.head()


,age,job,marital,education,default,balance,housing,loan,contact,day,...,job_unknown,job_retired,job_services,job_self_employed,job_unemployed,job_maid,job_student,married,single,divorced
0,58,management,married,tertiary,no,2143,yes,no,unknown,5,...,0,0,0,0,0,0,0,1,0,0
1,44,technician,single,secondary,no,29,yes,no,unknown,5,...,0,0,0,0,0,0,0,0,1,1
2,33,entrepreneur,married,secondary,no,2,yes,yes,unknown,5,...,0,0,0,0,0,0,0,1,0,0
3,47,blue-collar,married,unknown,no,1506,yes,no,unknown,5,...,0,0,0,0,0,0,0,1,0,0
4,33,unknown,single,unknown,no,1,no,no,unknown,5,...,1,0,0,0,0,0,0,0,1,1


In [63]:
bank_full.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45211 entries, 0 to 45210
Data columns (total 37 columns):
 #   Column                        Non-Null Count  Dtype 
---  ------                        --------------  ----- 
 0   age                           45211 non-null  int64 
 1   job                           45211 non-null  object
 2   marital                       45211 non-null  object
 3   education                     45211 non-null  object
 4   default                       45211 non-null  object
 5   balance                       45211 non-null  int64 
 6   housing                       45211 non-null  object
 7   loan                          45211 non-null  object
 8   contact                       45211 non-null  object
 9   day                           45211 non-null  int64 
 10  month                         45211 non-null  object
 11  duration                      45211 non-null  int64 
 12  campaign                      45211 non-null  int64 
 13  pdays           

In [64]:
x = bank_full.iloc[:,[18,19,20,21,22,23,24,25,26,27,28,29,31,32,33,34,35,36]].values
y = bank_full.iloc[:,17].values

In [65]:
LogReg = LogisticRegression()
LogReg.fit(x,y)
y_pred = LogReg.predict(x)

In [66]:
print(classification_report(y,y_pred))

              precision    recall  f1-score   support

           0       0.90      0.99      0.94     39922
           1       0.67      0.16      0.26      5289

    accuracy                           0.89     45211
   macro avg       0.79      0.58      0.60     45211
weighted avg       0.87      0.89      0.86     45211

